In [21]:
import numpy as np
import logging
import gzip
import matplotlib.pyplot as plt

PWD="/home/duzicman/projects/tau/statistical_machine_learning/logistic_classifier/datasets"
DATASET = {"labels": {"path" :PWD + "/train-labels-idx1-ubyte.gz", "offset":8}, "data": {"path": PWD + "/train-images-idx3-ubyte.gz", "offset":16}}
IMAGE_LENGTH = 28
IMAGE_SIZE = IMAGE_LENGTH ** 2


In [22]:
def load_dataset(file_path, offset):
    with gzip.open(file_path, 'rb') as data_file:
        data = np.frombuffer(data_file.read(), np.uint8, offset=offset)
    return data

x_sample = load_dataset(DATASET["data"]["path"], DATASET["data"]["offset"])
t_labels = load_dataset(DATASET["labels"]["path"], DATASET["labels"]["offset"])


In [23]:
DATA_SIZE = 1000
def one_hot_encoding(labels):
    label_ammount = np.max(labels) + 1
    return np.eye(label_ammount)[labels]

def format_dataset(x_sample, labels):
    labels = one_hot_encoding(labels)
    x_sample = np.reshape(x_sample, (-1, IMAGE_SIZE))
    x_sample = np.c_[x_sample, np.ones(x_sample.shape[0])]
    return x_sample, labels

x_sample, t_labels = format_dataset(x_sample, t_labels)
class_number = t_labels.shape[1]
#TODO remove partial data size
x_sample = x_sample[:DATA_SIZE] 
t_labels = t_labels[:DATA_SIZE]
 
x_sample = np.transpose(x_sample)

In [24]:
def show_image(image):
    pixels = np.reshape(image, (IMAGE_LENGTH, IMAGE_LENGTH))
    plt.imshow(pixels, cmap='gray')
    plt.show()
# show_image(features[0])
#TODO I added extra normilization pixel so it doesnt work noe
# print(labels[0])


In [33]:
def cross_entropy_loss(y_labels_probabilites, t_labels) -> float:
    natural_log = lambda x: np.log(x)
    y_logged = np.matrix.transpose(np.vectorize(natural_log)(y_labels_probabilites))
    return -np.trace(np.matmul(t_labels, y_logged))

def caculate_lables_probabilities(w_weights, x_sample):
    a_logits = np.matmul(np.transpose(x_sample), w_weights)
    exponent = lambda x: np.exp(x)
    exponent_logits = np.vectorize(exponent)(a_logits)
    row_summation = np.sum(exponent_logits, axis=1)[:, np.newaxis]
    y_labels_probabilities = exponent_logits /  row_summation
    return y_labels_probabilities

def cross_entropy_loss_gradient(y_label_probabilites, t_labels, x_sample):
    return np.matmul(x_sample, (y_label_probabilites - t_labels))

def classification_accuracy(y_label_probabilites, t_labels):
    classification = np.eye(y_label_probabilites.shape[0], y_label_probabilites.shape[1])[np.argmax(y_label_probabilites, axis=1)]
    return np.sum(np.all(t_labels == classification, axis=1)) / classification.shape[0]

#TODO rename probablity to a single name
#TODO start with random probabilities
w_weights = np.zeros((IMAGE_SIZE + 1, class_number))
INITIAL_LEARNING_RATE = 0.0000002
eta_learning_rate = INITIAL_LEARNING_RATE

In [34]:
def gradient_descent(w_initial_weights, x_sample, t_labels, eta_learning_rate, loss_epsilon):
    w_weights = w_initial_weights
    accuracies = []
    losses = []
    #TODO change stop conditiono to accuracies?
    while len(losses) < 2 or abs(losses[-1] - losses[-2]) > loss_epsilon:
        y_labels_probabilities = caculate_lables_probabilities(w_weights, x_sample)
        loss_gradient = cross_entropy_loss_gradient(y_labels_probabilities, t_labels, x_sample)
        accuracy = classification_accuracy(y_labels_probabilities, t_labels)
        loss = cross_entropy_loss(y_labels_probabilities, t_labels)
        losses.append(loss)
        accuracies.append(accuracy)
        w_weights -= loss_gradient * eta_learning_rate
    print("Loss {}".format(loss))
    print("Accuraccy {}".format(accuracy))
    print("Steps {}".format(len(losses)))

gradient_descent(np.zeros((IMAGE_SIZE + 1, class_number)), x_sample, t_labels, eta_learning_rate, 1)


Loss 538.3414365002342
Accuraccy 0.927
Steps 32
